# <b> Clean up </b>

## AutoReload

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Clean up

In [12]:
import boto3
from utils.ddb import ddb_handler
from utils.iam import iam_handler
from utils.ssm import parameter_store
from utils.lambda_func import lambda_handler
from utils.api_gateway import api_gateway_handler

In [13]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)
strPrefix = pm.get_params(key="PREFIX")

> API-Gateway and Lambda

In [4]:
lam_handler = lambda_handler(region_name=pm.get_params(key="".join([strPrefix, "REGION"])))
apig = api_gateway_handler(pm.get_params(key=strPrefix+"REGION"))
strLambdaName = pm.get_params(key=strPrefix+"LAMBDA-NAME")
strApiId = pm.get_params(key=strPrefix+"API-ID")

lam_handler.delete_function(strLambdaName)
apig._delete_rest_api(api_id=strApiId)

  lambda function: [API-GATEWAY-LAMBDA-DEMO-Lambda] is deleted successfully
  api-gateway: [5q95fu1g5d] is deleted successfully


> DynamoDB and RDS (Aurora)

In [5]:
ddb_inf_res = ddb_handler(strTableName=pm.get_params(strPrefix+"INFERENCE-RES-TABLE-NAME"))
ddb_inf_res.delete_table()

Table [API-GATEWAY-LAMBDA-DEMO-inference-results] is ACTIVE now.
Table [API-GATEWAY-LAMBDA-DEMO-inference-results] is deleted successfully.


>Role

In [9]:
iam = iam_handler()
listRoleNames = [strRoleName for strRoleName in iam.get_all_role_names() if strPrefix in strRoleName]
for strRoleName in listRoleNames:
    print (strRoleName)
    iam.delete_role(strRoleName=strRoleName)

API-GATEWAY-LAMBDA-DEMO-LambdaRole
{'ResponseMetadata': {'RequestId': 'c0dcfedc-7a43-4286-9306-88f6adeb98e9', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'c0dcfedc-7a43-4286-9306-88f6adeb98e9', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Fri, 09 Dec 2022 03:00:20 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole
{'ResponseMetadata': {'RequestId': '2010cd23-21cf-4413-bb89-7982801ec2fe', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '2010cd23-21cf-4413-bb89-7982801ec2fe', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Fri, 09 Dec 2022 03:00:20 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AmazonDynamoDBFullAccess
{'ResponseMetadata': {'RequestId': '9eb0898d-1ca0-4495-8fd1-fd823d784195', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '9eb0898d-1ca0-4495-8fd1-fd823d784195', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Fri, 09 Dec 2022 03:00:21 GMT'}, 'R

>Parameter store

In [15]:
listParams = pm.get_all_params()
listParams = [strParam for strParam in listParams if strPrefix in strParam] + [strPrefix]
pm.delete_param(listParams)

  parameters: ['API-GATEWAY-LAMBDA-DEMO-'] is deleted successfully
